# Tarea U2S4-ComparaciónOCR

## easyOCR

### Instalacion de easyOCR

In [1]:
# !pip install easyOCR

### Comprobar version de easyOCR

In [2]:
!pip show easyOCR

Name: easyocr
Version: 1.7.2
Summary: End-to-End Multi-Lingual Optical Character Recognition (OCR) Solution
Home-page: https://github.com/jaidedai/easyocr
Author: Rakpong Kittinaradorn
Author-email: r.kittinaradorn@gmail.com
License: Apache License 2.0
Location: /opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages
Requires: ninja, numpy, opencv-python-headless, Pillow, pyclipper, python-bidi, PyYAML, scikit-image, scipy, Shapely, torch, torchvision
Required-by: 


### Probar con una imagen easyOCR (presicion y tiempo de inferencia)

In [3]:
import easyocr
import time
import warnings
import os

# Suprimir warnings y mensajes de info
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configuración
image_path = 'dataset/1755800220_0.9_(48, 104, 3)_BYJ-567.jpg'
expected_text = "BYJ-567"

# Inicializar easyOCR
reader = easyocr.Reader(['es', 'en'], gpu=False, verbose=False)

# Realizar OCR y medir tiempo
start_time = time.time()
results = reader.readtext(image_path)
inference_time = time.time() - start_time

# Filtrar detecciones que tengan exactamente 7 caracteres
valid_detections = [text for _, text, conf in results if len(text.replace(' ', '').replace('-', '')) == 6 and len(text) == 7]
detected_text = valid_detections[0] if valid_detections else ""

# Calcular precisión
precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"

# Mostrar resultados en tabla
print("\n" + "=" * 120)
print("RESULTADOS easyOCR")
print("=" * 120)
print(f"{'Imagen':<45} | {'Texto Detectado':<15} | {'Texto Esperado':<15} | {'Tiempo (s)':<12} | {'Precisión':<10} | {'Resultado':<12}")
print("-" * 120)
print(f"{image_path:<45} | {detected_text:<15} | {expected_text:<15} | {inference_time:<12.4f} | {precision:<10.1f}% | {resultado:<12}")
print("=" * 120)


RESULTADOS easyOCR
Imagen                                        | Texto Detectado | Texto Esperado  | Tiempo (s)   | Precisión  | Resultado   
------------------------------------------------------------------------------------------------------------------------
dataset/1755800220_0.9_(48, 104, 3)_BYJ-567.jpg | BYJ-567         | BYJ-567         | 0.0734       | 100.0     % | ✅ Correcto  


### Probar con el dataset easyOCR (presicion y tiempo de inferencia)

In [4]:
import easyocr
import time
import warnings
import os
import glob
import re
import csv

# Suprimir warnings y mensajes de info
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configuración
dataset_path = 'dataset/'
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
csv_filename = 'resultados_easyOCR/resultados_easyocr.csv'

# Obtener todas las imágenes del dataset
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(dataset_path, ext)))

# Inicializar easyOCR
print("Inicializando easyOCR...")
reader = easyocr.Reader(['es', 'en'], gpu=False, verbose=False)

# Almacenar resultados
results_data = []
total_time = 0
correct_count = 0
total_images = len(image_files)

# Procesar cada imagen
for image_path in sorted(image_files):
    
    # Extraer el texto esperado del nombre del archivo
    filename = os.path.basename(image_path)
    match = re.search(r'_([A-Z0-9-]+)\.(jpg|jpeg|png|bmp)$', filename, re.IGNORECASE)
    expected_text = match.group(1) if match else "UNKNOWN"
    
    # Realizar OCR y medir tiempo
    start_time = time.time()
    results = reader.readtext(image_path)
    inference_time = time.time() - start_time
    total_time += inference_time

    # Filtrar detecciones que tengan exactamente 7 caracteres
    valid_detections = [text for _, text, conf in results if len(text.replace(' ', '').replace('-', '')) == 6 and len(text) == 7]
    detected_text = valid_detections[0] if valid_detections else ""

    # Calcular precisión
    precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
    resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"
    
    if precision == 100.0:
        correct_count += 1

    # Guardar resultado
    results_data.append({
        'Imagen': image_path,
        'Texto Detectado': detected_text,
        'Texto Esperado': expected_text,
        'Tiempo (s)': f"{inference_time:.4f}",
        'Precisión (%)': f"{precision:.1f}",
        'Resultado': resultado
    })

    print(f"Procesado: {filename} | Detectado: {detected_text} | Esperado: {expected_text} | Tiempo: {inference_time:.4f}s | Precisión: {precision:.1f}% | {resultado}")

# Guardar resultados en CSV
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Imagen', 'Texto Detectado', 'Texto Esperado', 'Tiempo (s)', 'Precisión (%)', 'Resultado']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(results_data)
    
    # Agregar línea vacía
    writer.writerow({})
    
    # Agregar resumen final
    incorrect_count = total_images - correct_count
    writer.writerow({
        'Imagen': '=== RESUMEN FINAL ===',
        'Texto Detectado': '',
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Total de imágenes',
        'Texto Detectado': str(total_images),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Correctos',
        'Texto Detectado': str(correct_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(correct_count/total_images)*100:.1f}",
        'Resultado': '✅'
    })
    writer.writerow({
        'Imagen': 'Incorrectos',
        'Texto Detectado': str(incorrect_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(incorrect_count/total_images)*100:.1f}",
        'Resultado': '❌'
    })
    writer.writerow({
        'Imagen': 'Tiempo total',
        'Texto Detectado': f"{total_time:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Tiempo promedio',
        'Texto Detectado': f"{total_time/total_images:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })

print(f"\n✅ Resultados guardados en: {csv_filename}")
print(f"📊 Resumen: {correct_count}/{total_images} correctos ({(correct_count/total_images)*100:.1f}%)")
print(f"⏱️  Tiempo total: {total_time:.4f}s | Tiempo promedio: {total_time/total_images:.4f}s")

Inicializando easyOCR...
Procesado: 1755800220_0.9_(48, 104, 3)_BYJ-567.jpg | Detectado: BYJ-567 | Esperado: BYJ-567 | Tiempo: 0.0307s | Precisión: 100.0% | ✅ Correcto
Procesado: 1755800225_0.91_(44, 95, 3)_V71-023.jpg | Detectado:  | Esperado: V71-023 | Tiempo: 0.0253s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800256_0.9_(49, 104, 3)_Z4X-018.jpg | Detectado: 74X-018 | Esperado: Z4X-018 | Tiempo: 0.0217s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800279_0.93_(64, 113, 3)_A8D-757.jpg | Detectado: [80-757 | Esperado: A8D-757 | Tiempo: 0.0213s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800317_0.9_(45, 93, 3)_VAJ-632.jpg | Detectado:  | Esperado: VAJ-632 | Tiempo: 0.0195s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800323_0.9_(52, 105, 3)_CRO-577.jpg | Detectado: [RO-577 | Esperado: CRO-577 | Tiempo: 0.0370s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800341_0.9_(44, 91, 3)_V8U-524.jpg | Detectado: 18u-524 | Esperado: V8U-524 | Tiempo: 0.0201s | Precisión: 0.0% | ❌ 

## TesseractOCR

### Instalacion de TesseractOCR


In [5]:
# !pip install pytesseract pillow

### Comprobar version de TesseractOCR

In [6]:
!pip show pytesseract

Name: pytesseract
Version: 0.3.13
Summary: Python-tesseract is a python wrapper for Google's Tesseract-OCR
Home-page: https://github.com/madmaze/pytesseract
Author: Samuel Hoffstaetter
Author-email: samuel@hoffstaetter.com
License: Apache License 2.0
Location: /opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages
Requires: packaging, Pillow
Required-by: 


### Probar con una imagen TesseractOCR (presicion y tiempo de inferencia)

In [7]:
import pytesseract
import cv2
import time
import os
import re

# Configuración
image_path = 'dataset/1755801131_0.9_(56, 113, 3)_A9K-715.jpg'

# Extraer texto esperado del nombre del archivo
filename = os.path.basename(image_path)
match = re.search(r'_([A-Z0-9-]+)\.(jpg|jpeg|png|bmp)$', filename, re.IGNORECASE)
expected_text = match.group(1) if match else "UNKNOWN"

# Configuración de Tesseract para placas
config_tesseract = '-l spa --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-'

# Leer y preprocesar imagen
imagen = cv2.imread(image_path)
if imagen is None:
    print(f"❌ Error: No se pudo cargar la imagen en {image_path}")
    exit()

gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Realizar OCR y medir tiempo
start_time = time.time()
texto = pytesseract.image_to_string(thresh, config=config_tesseract)
inference_time = time.time() - start_time

# Limpiar texto detectado
detected_text = texto.strip().replace('\n', ' ').replace('\f', '')

# Calcular precisión
precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"

# Mostrar resultados en tabla
print("\n" + "=" * 120)
print("RESULTADOS TesseractOCR")
print("=" * 120)
print(f"{'Imagen':<45} | {'Texto Detectado':<15} | {'Texto Esperado':<15} | {'Tiempo (s)':<12} | {'Precisión':<10} | {'Resultado':<12}")
print("-" * 120)
print(f"{image_path:<45} | {detected_text:<15} | {expected_text:<15} | {inference_time:<12.4f} | {precision:<10.1f}% | {resultado:<12}")
print("=" * 120)


RESULTADOS TesseractOCR
Imagen                                        | Texto Detectado | Texto Esperado  | Tiempo (s)   | Precisión  | Resultado   
------------------------------------------------------------------------------------------------------------------------
dataset/1755801131_0.9_(56, 113, 3)_A9K-715.jpg | A9K-715         | A9K-715         | 0.0533       | 100.0     % | ✅ Correcto  


### Probar con el dataset TesseractOCR (presicion y tiempo de inferencia)

In [8]:
import pytesseract
import cv2
import time
import warnings
import os
import glob
import re
import csv

# Configuración
dataset_path = 'dataset/'
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
csv_filename = 'resultados_tesseractOCR/resultados_tesseractocr.csv'

# Crear carpeta de salida si no existe
os.makedirs('resultados_tesseractOCR', exist_ok=True)

# Obtener todas las imágenes del dataset
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(dataset_path, ext)))

# Configuración de Tesseract para placas
config_tesseract = '-l spa --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789-'

print("Inicializando TesseractOCR...")
print(f"Total de imágenes a procesar: {len(image_files)}\n")

# Almacenar resultados
results_data = []
total_time = 0
correct_count = 0
total_images = len(image_files)

# Procesar cada imagen
for image_path in sorted(image_files):
    
    # Extraer el texto esperado del nombre del archivo
    filename = os.path.basename(image_path)
    match = re.search(r'_([A-Z0-9-]+)\.(jpg|jpeg|png|bmp)$', filename, re.IGNORECASE)
    expected_text = match.group(1) if match else "UNKNOWN"
    
    # Leer y preprocesar imagen
    imagen = cv2.imread(image_path)
    
    if imagen is None:
        detected_text = ""
        inference_time = 0
        print(f"❌ Error al cargar: {filename}")
    else:
        # Preprocesamiento
        gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gris, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # Realizar OCR y medir tiempo
        start_time = time.time()
        texto = pytesseract.image_to_string(thresh, config=config_tesseract)
        inference_time = time.time() - start_time
        total_time += inference_time
        
        # Limpiar texto detectado
        detected_text = texto.strip().replace('\n', ' ').replace('\f', '')
    
    # Calcular precisión
    precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
    resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"
    
    if precision == 100.0:
        correct_count += 1
    
    # Guardar resultado
    results_data.append({
        'Imagen': image_path,
        'Texto Detectado': detected_text,
        'Texto Esperado': expected_text,
        'Tiempo (s)': f"{inference_time:.4f}",
        'Precisión (%)': f"{precision:.1f}",
        'Resultado': resultado
    })
    
    print(f"Procesado: {filename} | Detectado: {detected_text} | Esperado: {expected_text} | Tiempo: {inference_time:.4f}s | Precisión: {precision:.1f}% | {resultado}")

# Guardar resultados en CSV
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Imagen', 'Texto Detectado', 'Texto Esperado', 'Tiempo (s)', 'Precisión (%)', 'Resultado']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(results_data)
    
    # Agregar línea vacía
    writer.writerow({})
    
    # Agregar resumen final
    incorrect_count = total_images - correct_count
    writer.writerow({
        'Imagen': '=== RESUMEN FINAL ===',
        'Texto Detectado': '',
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Total de imágenes',
        'Texto Detectado': str(total_images),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Correctos',
        'Texto Detectado': str(correct_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(correct_count/total_images)*100:.1f}",
        'Resultado': '✅'
    })
    writer.writerow({
        'Imagen': 'Incorrectos',
        'Texto Detectado': str(incorrect_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(incorrect_count/total_images)*100:.1f}",
        'Resultado': '❌'
    })
    writer.writerow({
        'Imagen': 'Tiempo total',
        'Texto Detectado': f"{total_time:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Tiempo promedio',
        'Texto Detectado': f"{total_time/total_images:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })

print(f"\n✅ Resultados guardados en: {csv_filename}")
print(f"📊 Resumen: {correct_count}/{total_images} correctos ({(correct_count/total_images)*100:.1f}%)")
print(f"⏱️  Tiempo total: {total_time:.4f}s | Tiempo promedio: {total_time/total_images:.4f}s")

Inicializando TesseractOCR...
Total de imágenes a procesar: 500

Procesado: 1755800220_0.9_(48, 104, 3)_BYJ-567.jpg | Detectado:  | Esperado: BYJ-567 | Tiempo: 0.0504s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800225_0.91_(44, 95, 3)_V71-023.jpg | Detectado:  | Esperado: V71-023 | Tiempo: 0.0570s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800256_0.9_(49, 104, 3)_Z4X-018.jpg | Detectado:  | Esperado: Z4X-018 | Tiempo: 0.0759s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800279_0.93_(64, 113, 3)_A8D-757.jpg | Detectado: 80-757 | Esperado: A8D-757 | Tiempo: 0.0562s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800317_0.9_(45, 93, 3)_VAJ-632.jpg | Detectado:  | Esperado: VAJ-632 | Tiempo: 0.0563s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800323_0.9_(52, 105, 3)_CRO-577.jpg | Detectado:  | Esperado: CRO-577 | Tiempo: 0.0559s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800341_0.9_(44, 91, 3)_V8U-524.jpg | Detectado:  | Esperado: V8U-524 | Tiempo: 0.0510s | Precisión

## PaddleOCR

### Instalacion de PaddleOCR


In [9]:
# !pip install paddleocr

### Comprobar version de PaddleOCR

In [10]:
!pip show paddleocr

Name: paddleocr
Version: 3.2.0
Summary: Awesome multilingual OCR and document parsing toolkits based on PaddlePaddle
Home-page: https://github.com/PaddlePaddle/PaddleOCR
Author: 
Author-email: PaddlePaddle <Paddle-better@baidu.com>
License: Apache License 2.0
Location: /opt/homebrew/Caskroom/miniforge/base/envs/torch2/lib/python3.10/site-packages
Requires: paddlex, PyYAML, typing-extensions
Required-by: 


### Probar con una imagen PaddleOCR (presicion y tiempo de inferencia)

In [11]:
from paddleocr import PaddleOCR
import time
import warnings
import os
import re

# Suprimir warnings y mensajes de info
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configuración
image_path = 'dataset/1755800317_0.9_(45, 93, 3)_VAJ-632.jpg'

# Extraer texto esperado del nombre del archivo
filename = os.path.basename(image_path)
match = re.search(r'_([A-Z0-9-]+)\.(jpg|jpeg|png|bmp)$', filename, re.IGNORECASE)
expected_text = match.group(1) if match else "UNKNOWN"

# Inicializar PaddleOCR (sin show_log)
ocr = PaddleOCR(lang="es", use_angle_cls=True)

# Realizar OCR y medir tiempo
start_time = time.time()
resultados = ocr.ocr(image_path)
inference_time = time.time() - start_time

# Extraer textos detectados
detected_texts = []

for bloque in resultados:
    for texto, confianza in zip(bloque['rec_texts'], bloque['rec_scores']):
        detected_texts.append(texto)

valid_detections = [text for text in detected_texts]
detected_text = valid_detections[0] if valid_detections else ""

# Calcular precisión
precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"

# Mostrar resultados en tabla
print("\n" + "=" * 120)
print("RESULTADOS PaddleOCR")
print("=" * 120)
print(f"{'Imagen':<45} | {'Texto Detectado':<15} | {'Texto Esperado':<15} | {'Tiempo (s)':<12} | {'Precisión':<10} | {'Resultado':<12}")
print("-" * 120)
print(f"{image_path:<45} | {detected_text:<15} | {expected_text:<15} | {inference_time:<12.4f} | {precision:<10.1f}% | {resultado:<12}")
print("=" * 120)

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/fransp


RESULTADOS PaddleOCR
Imagen                                        | Texto Detectado | Texto Esperado  | Tiempo (s)   | Precisión  | Resultado   
------------------------------------------------------------------------------------------------------------------------
dataset/1755800317_0.9_(45, 93, 3)_VAJ-632.jpg | VAJ-632         | VAJ-632         | 0.3854       | 100.0     % | ✅ Correcto  


### Probar con el dataset PaddleOCR (presicion y tiempo de inferencia)

In [12]:
from paddleocr import PaddleOCR
import time
import warnings
import os
import glob
import re
import csv

# Suprimir warnings y mensajes de info
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Configuración
dataset_path = 'dataset/'
image_extensions = ['*.jpg', '*.jpeg', '*.png', '*.bmp']
csv_filename = 'resultados_paddleOCR/resultados_paddleocr.csv'

# Crear carpeta de salida si no existe
os.makedirs('resultados_paddleOCR', exist_ok=True)

# Obtener todas las imágenes del dataset
image_files = []
for ext in image_extensions:
    image_files.extend(glob.glob(os.path.join(dataset_path, ext)))

print("Inicializando PaddleOCR...")
# Inicializar PaddleOCR
ocr = PaddleOCR(lang="es", use_angle_cls=True)
print(f"Total de imágenes a procesar: {len(image_files)}\n")

# Almacenar resultados
results_data = []
total_time = 0
correct_count = 0
total_images = len(image_files)

# Procesar cada imagen
for image_path in sorted(image_files):
    
    # Extraer el texto esperado del nombre del archivo
    filename = os.path.basename(image_path)
    match = re.search(r'_([A-Z0-9-]+)\.(jpg|jpeg|png|bmp)$', filename, re.IGNORECASE)
    expected_text = match.group(1) if match else "UNKNOWN"
    
    # Realizar OCR y medir tiempo
    start_time = time.time()
    resultados = ocr.ocr(image_path)
    inference_time = time.time() - start_time
    total_time += inference_time
    
    # Extraer textos detectados
    detected_texts = []
    for bloque in resultados:
        for texto, confianza in zip(bloque['rec_texts'], bloque['rec_scores']):
            detected_texts.append(texto)

    valid_detections = [text for text in detected_texts]
    detected_text = valid_detections[0] if valid_detections else ""
    
    # Calcular precisión
    precision = 100.0 if expected_text.lower() == detected_text.lower() else 0.0
    resultado = "✅ Correcto" if precision == 100.0 else "❌ Incorrecto"
    
    if precision == 100.0:
        correct_count += 1
    
    # Guardar resultado
    results_data.append({
        'Imagen': image_path,
        'Texto Detectado': detected_text,
        'Texto Esperado': expected_text,
        'Tiempo (s)': f"{inference_time:.4f}",
        'Precisión (%)': f"{precision:.1f}",
        'Resultado': resultado
    })
    
    print(f"Procesado: {filename} | Detectado: {detected_text} | Esperado: {expected_text} | Tiempo: {inference_time:.4f}s | Precisión: {precision:.1f}% | {resultado}")

# Guardar resultados en CSV
with open(csv_filename, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Imagen', 'Texto Detectado', 'Texto Esperado', 'Tiempo (s)', 'Precisión (%)', 'Resultado']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    writer.writerows(results_data)
    
    # Agregar línea vacía
    writer.writerow({})
    
    # Agregar resumen final
    incorrect_count = total_images - correct_count
    writer.writerow({
        'Imagen': '=== RESUMEN FINAL ===',
        'Texto Detectado': '',
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Total de imágenes',
        'Texto Detectado': str(total_images),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Correctos',
        'Texto Detectado': str(correct_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(correct_count/total_images)*100:.1f}",
        'Resultado': '✅'
    })
    writer.writerow({
        'Imagen': 'Incorrectos',
        'Texto Detectado': str(incorrect_count),
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': f"{(incorrect_count/total_images)*100:.1f}",
        'Resultado': '❌'
    })
    writer.writerow({
        'Imagen': 'Tiempo total',
        'Texto Detectado': f"{total_time:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })
    writer.writerow({
        'Imagen': 'Tiempo promedio',
        'Texto Detectado': f"{total_time/total_images:.4f}s",
        'Texto Esperado': '',
        'Tiempo (s)': '',
        'Precisión (%)': '',
        'Resultado': ''
    })

print(f"\n✅ Resultados guardados en: {csv_filename}")
print(f"📊 Resumen: {correct_count}/{total_images} correctos ({(correct_count/total_images)*100:.1f}%)")
print(f"⏱️  Tiempo total: {total_time:.4f}s | Tiempo promedio: {total_time/total_images:.4f}s")

Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/UVDoc`.
Creating model: ('PP-LCNet_x1_0_textline_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-LCNet_x1_0_textline_ori`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/franspaxi/.paddlex/official_models/PP-OCRv5_server_det`.
Creating model: ('latin_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `/Users/fransp

Inicializando PaddleOCR...
Total de imágenes a procesar: 500

Procesado: 1755800220_0.9_(48, 104, 3)_BYJ-567.jpg | Detectado: BYJ-56 | Esperado: BYJ-567 | Tiempo: 0.3641s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800225_0.91_(44, 95, 3)_V71-023.jpg | Detectado: V71-0 | Esperado: V71-023 | Tiempo: 0.2566s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800256_0.9_(49, 104, 3)_Z4X-018.jpg | Detectado: 4X-0 | Esperado: Z4X-018 | Tiempo: 0.2568s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800279_0.93_(64, 113, 3)_A8D-757.jpg | Detectado: A8D-75 | Esperado: A8D-757 | Tiempo: 0.2629s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800317_0.9_(45, 93, 3)_VAJ-632.jpg | Detectado: VAJ-632 | Esperado: VAJ-632 | Tiempo: 0.2622s | Precisión: 100.0% | ✅ Correcto
Procesado: 1755800323_0.9_(52, 105, 3)_CRO-577.jpg | Detectado: CRO-57 | Esperado: CRO-577 | Tiempo: 0.2592s | Precisión: 0.0% | ❌ Incorrecto
Procesado: 1755800341_0.9_(44, 91, 3)_V8U-524.jpg | Detectado: V8U-52 | Esperado: V8U-52

## Resultado finales (ver el .txt en la carpeta resultados_finales)